In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import pandas as pd

In [ ]:
#Reading the Training data (virtually generated) from a file 
#Input data
x_data = pd.read_csv('x_rec_16_data_5.csv')
x_data=x_data.values[:,1:]
print("read_x_data")
#Output data
y_data = pd.read_csv('y_rec_16_data_5.csv')
y_data=y_data.values[:,1:]
print("read_y_data")
print(x_data[:2,:])
print(y_data[:2,:])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(x_data, y_data)
print(np.shape(X_train), np.shape(y_train), "training data size")
print(np.shape(X_val), np.shape(y_val),"Validation data size")
#print(X_train[0:9,:])

scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

scaler_x.fit(X_train)
xtrain_scale=scaler_x.transform(X_train)

scaler_x.fit(X_val)
xval_scale=scaler_x.transform(X_val)

scaler_y.fit(y_train)
ytrain_scale=scaler_y.transform(y_train)

scaler_y.fit(y_val)
yval_scale=scaler_y.transform(y_val)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense
from numpy import loadtxt
from keras.models import load_model


input_layer = Input(shape=(48,))
hidden_layer_1 = Dense(units = 700, activation = 'relu')(input_layer)
hidden_layer_2 = Dense(units = 700, activation = 'relu')(hidden_layer_1)
output_layer = Dense(units = 7, activation = 'linear')(hidden_layer_2)
model = Model(inputs = input_layer, outputs = output_layer, name = 'sensor_concept_nn')
#opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='mse', optimizer=opt)
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Display the model
model.summary()

In [ ]:
history = model.fit(xtrain_scale, ytrain_scale, epochs=30, verbose=2, validation_data=(xval_scale,yval_scale),callbacks=[callback])


In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
#plt.ylim(0.07,0.15)
plt.show()

In [ ]:

#Creating a Callback subclass that stores each epoch prediction
class prediction_history(tf.keras.callbacks.Callback):
    def __init__(self):
        self.predhis = []
    def on_epoch_end(self, epoch, logs={}):
        self.predhis.append(model.predict(xval_scale))

#Calling the subclass
predictions=prediction_history()

#Executing the model.fit of the neural network
#model.fit(X=predictor_train, y=target_train, nb_epoch=2, batch_size=batch,validation_split=0.1,callbacks=[predictions]) 
history = model.fit(xtrain_scale, ytrain_scale, epochs=30, verbose=2, validation_data=(xval_scale,yval_scale),callbacks=[callback,predictions])
#Printing the prediction history
#print (predictions.predhis)

In [ ]:
check_data=predictions.predhis
np.shape(check_data)
err_sincos_mean=[]

for i in range(0,20):
    err_sincos = (check_data[i][:][2]**2+check_data[i][:][3]**2)-1
    err_sincos_mean.append(np.mean(err_sincos))
print(err_sincos_mean)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(err_sincos_mean)
plt.title('Model Loss with sincos')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val','sincos'], loc='upper left')
#plt.ylim(0.07,0.15)
plt.show()

In [ ]:
asd = np.array(history.history['val_loss'])
print(asd)

plt.plot(asd)
plt.plot(err_sincos_mean)
plt.show()

In [ ]:
fig, axs = plt.subplots(2,sharex=True)
fig.suptitle('Validation Loss vs. Error in Sin and Cos predictions ')
axs[0].plot(asd)
axs[1].plot(err_sincos_mean)
ax

In [ ]:
plt.plot(asd/err_sincos_mean)
plt.ylim(-0.5,0.5)
plt.show()
print(np.mean(asd/err_sincos_mean))
plt.xlabel('epochs')
plt.ylabel('error in Sin and Cos prediction')
plt.title('epochs vs. prediction error in Sin and Cos')

### err_sincos_mean is sin**2(beta)+cos**2(beat)-1
### asd is validation loss of the model

# asd = constant * err_sincos_mean

### the constant in this case is -0.2 
